<a href="https://colab.research.google.com/github/bell-boy/SeizureSense/blob/main/EEGMamba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imports

In [1]:
!pip install mne
!pip install einops
!pip install
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json
from torch import tensor
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import argparse
import os
gpus = [0]
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, gpus))
import numpy as np
import math
import glob
import random
import itertools
import datetime
import time
import datetime
import sys
import scipy.io
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

'''from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchsummary import summary
import torch.autograd as autograd
from torchvision.models import vgg19'''

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn.init as init

from torch.utils.data import Dataset
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
# from common_spatial_pattern import csp
!pip install causal-conv1d>=1.1.0
!pip install mamba-ssm
import torch
from mamba_ssm import Mamba

ERROR: You must give at least one requirement to install (see "pip help install")


# Model architecture mamba block

In [2]:
class MambaBlock(nn.Module):
    def __init__(self, dim, d_state=16, d_conv=4, expand=2):
        super(MambaBlock, self).__init__()
        self.mamba = Mamba(
            d_model=dim,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        ).to("cuda")

    def forward(self, x):
        return self.mamba(x)


In [3]:
class StackedMambaModel(nn.Module):
    def __init__(self, num_layers, dim, d_state=16, d_conv=4, expand=2):
        super(StackedMambaModel, self).__init__()
        self.layers = nn.ModuleList([MambaBlock(dim, d_state, d_conv, expand) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [6]:
class EEGMamba(nn.Module):
    def __init__(self, num_channels, num_layers, dim, d_state=16, d_conv=4, expand=2):
        super(EEGMamba, self).__init__()
        self.channel_models = nn.ModuleList([StackedMambaModel(num_layers, dim, d_state, d_conv, expand) for _ in range(num_channels)])

    def forward(self, x):
        batch_size = x.size(0)
        channel_outputs = []

        # Iterate over each channel model and corresponding channel data
        for i in range(x.size(1)):
            channel_data = x[:, i, :, :]  # Extract data for the i-th channel
            model = self.channel_models[i]
            output = model(channel_data)
            averaged_output = torch.mean(output, dim=1, keepdim=True)
            channel_outputs.append(averaged_output)

        # Stack the channel outputs along the correct dimension
        combined_output = torch.stack(channel_outputs, dim=1)
        return combined_output

In [7]:
#just testing it on a random pytorch tensor for now
batch_size = 16

num_channels = 23
sequence_length = 15360
dim =1  # This is where `dim` is set

data = torch.randn(batch_size, num_channels, sequence_length, dim).to("cuda")
model = EEGMamba(num_channels, num_layers=4, dim=dim, d_state=16, d_conv=4, expand=2).to("cuda")
output = model(data)
print(output.shape)  # Output shape: (batch_size, num_channels, 1,dim

torch.Size([16, 23, 1, 1])


In [ ]:
torch.cuda.empty_cache()


# preprocessing for testing

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
mne.set_log_level("ERROR")
raw = mne.io.read_raw_edf("chb02_19.edf", preload=True)

def preprocessing(raw):

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60,120)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)

    raw.notch_filter(freqs=freqs, picks=eeg_picks)  # Apply notch filter

    numpy_array=raw.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return(segment_tensor)